# Constrained turbulence

This notebook contains a demonstration of how the `pyconturb` package can be easily used to generate turbulence that is constrained by one or more time series at a point in space.

The demo uses pre-generated time series that were simulated at a series of vertically spaced points. The `gen_turb` function is then used with these time series as constraints to result in a turbulence box that follows IEC 61400-1 Ed. 3 specifications but is correlated with the constraints. In other words, the coherence between the constraining time series and the simulated time series also follow IEC specifications.

It is worth noting that other models for the spectral magnitudes or coherences can be used. This demo is only meant as a proof-of-concept.


This example has the following sections:  

* [Preliminaries](#Preliminaries:-importing-functions)  

* [Constraining time series](#Constraining-time-series)  

* [Inputs to constrained turbulence](#Inputs-to-constrained-turbulence)  

* [Generate constrained turbulence](#Generate-constrained-turbulence)  

## Preliminaries: importing functions

We first set a few notebook-specific functions/variables and then import functions from the `pyconturb` package.

In [ ]:
%matplotlib inline
import os

import matplotlib.pyplot as plt  # matplotlib for some plotting
import numpy as np  # numeric python functions
import pandas as pd  # need this to load our data from the csv files

from pyconturb.simulation import gen_turb  # generate turbulence function
from pyconturb._utils import gen_spat_grid  # useful helper function

data_dir = os.path.join(os.path.abspath('.'), 'data')

## Constraining time series

Let us first load the spatial and turbulence dataframes of the constraining data to get an idea of what constraints will be present. We can also do some basic visualization to get a better idea of what these constraining time series look like.

In [ ]:
con_spat_path = os.path.join(data_dir, 'conturb_spat.csv')  # path to spat csv
con_spat_df = pd.read_csv(con_spat_path)  # spatial info
con_turb_path = os.path.join(data_dir, 'conturb.csv')  # path to saved constrain turbulence
con_turb_df = pd.read_csv(con_turb_path).set_index('time')  # turbulence
dt = con_turb_df.index[1]  # get sample time from constr
T = con_turb_df.index[-1] + dt  # get length of sim

Let's look at how these spatial and turbulence dataframes for the constraining data need to be structured.

In [ ]:
con_spat_df.head()

In [ ]:
con_turb_df.head()

So, we can see that the format of these dataframes are very similar to what we saw in the unconstrained example. In particular, the spatial dataframe has 5 columns (turbulence component, point identifier and spatial location) and the turbulence dataframe has columns corresponding to the turbulence component/point ID and rows that correspond to the time step.

We can plot the points to visualize the locations of the constrainting points in space.

In [ ]:
spat_pts = con_spat_df[['p_id', 'y', 'z']].drop_duplicates()
[plt.scatter(spat_pts.loc[i, 'y'], spat_pts.loc[i, 'z'],
             label=f'p{spat_pts.loc[i, "p_id"]}') for i in spat_pts.index]
plt.legend(); plt.ylabel('height [m]'); plt.xticks([]);

It is apparant that the constraining time series are located at a series of vertical heights, exactly as we would have if we were using met mast data. Additionally, the numbering goes from `p0` at the lowest height to `p5` at the highest.

Now let's visualize the constraining time series.

In [ ]:
ax = con_turb_df.filter(regex='u_', axis=1).plot(lw=0.75)  # subselect long. wind component
ax.set_ylabel('longitudinal wind speed [m/s]');
[print(x) for x in con_turb_df.filter(regex='u_', axis=1).mean()];  # print mean values

We can see an increase in the mean wind speed for higher points, which is as expected.

Now, let's simulate a turbulence box.

## Inputs to constrained turbulence

The first step is to define the spatial information for the desired turbulence box and the related parameters for the turbulence generation technique. In this case we will use the default IEC 61400-1 Ed. 3 simulation procedures (Kaimal Spectrum with Exponential Coherence). By default it will utilize the mean wind speed profile from IEC 61400-1 Ed. 3, which is a power law with an exponent of 0.2.

In [ ]:
y = np.linspace(-10, 10, 5)  # lateral components of turbulent grid
z = np.linspace(35, 115, 9)  # vertical components of turbulent grid
kwargs = {'u_hub': 10, 'turb_class': 'B', 'z_hub': 75,  # necessary keyword arguments for IEC turbulence
          'T': T, 'dt': dt}  # simulation length (s) and time step (s)

This function below generates the actual spatial data. It assumes we want all three turbulence components at each spatial location.

In [ ]:
spat_df = gen_spat_grid(y, z)  # create our spatial pandas dataframe. Columns are k, p_id x, y, and z.

## Generate constrained turbulence

The actual constrained turbulence generation is now just a one-line function. Note that the `scale=True` option scales the theoretical magnitudes before correlation to prevent and loss/gain of spectral energy due to the discretization of the theoretical spectra.

In [ ]:
con_data = {'con_spat_df': con_spat_df,
            'con_turb_df': con_turb_df}  # assemble spatial and time of constraints
sim_turb_df = gen_turb(spat_df, con_data=con_data, **kwargs)

We can plot the result easily because it is a pandas dataframe.

In [ ]:
sim_turb_df.filter(regex='_p0', axis=1).plot();  # just p0

Or we can check out statistics of $u$ by height:

In [ ]:
stats = sim_turb_df.filter(regex='u_', axis=1).describe().loc[['mean', 'std']]
# plot
plt.clf(); plt.subplot(1, 2, 1);
plt.scatter(stats.loc['mean'], spat_df[spat_df.k==0].z.values, label='Mean profile')
plt.legend()
plt.subplot(1, 2, 2)
plt.scatter(stats.loc['std'], spat_df[spat_df.k==0].z.values, label='Std dev')
plt.legend();